# Prep
This notebook will prepare the systems of varying sizes starting from the MD snapshot which contains 54,000 atoms.

In [ ]:
from os.path import join
outdir = "inp"
geom = "base"
natoms = 54  # number of atoms in a molecule

Sizes to generate.

In [ ]:
cutoffs = [1]
cutoffs += range(2, 18, 2)
cutoffs += range(18, 42, 4)
cutoffs += range(42, 122, 8)
cutoffs += range(122, 282, 16)
cutoffs += range(282, 602, 32)
cutoffs += range(602, 1000, 64)
cutoffs += [1000]

ranges = {"easy": 42 - 4, "medium": 122 - 8, 
          "hard": 282 - 16, "very hard": 602-32,
          "extreme": 1000}

Read in the starting file.

In [ ]:
from BigDFT.IO import XYZReader
from BigDFT.Systems import System
from BigDFT.Fragments import Fragment

sys = System()

with XYZReader(join(outdir, geom + ".xyz")) as ifile:
    for i, at in enumerate(ifile):
        fid = "SOL:" + str(int((i)/natoms))
        if fid not in sys:
            sys[fid] = Fragment()
        sys[fid].append(at)

Order the fragments by distance to the centroid.

In [ ]:
from numpy import array
from numpy.linalg import norm

distances = {}
cent = sys.centroid
for fragid, frag in sys.items():
    distances[fragid] = norm(array(frag.centroid) - array(cent))

In [ ]:
newsys = System()
for i, fragid in enumerate(sorted(list(distances), key=distances.get)):
    if i == 0:
        newsys["TAR:0"] = sys[fragid]
    else:
        newsys["SOL:" + str(i)] = sys[fragid]

Generate.

In [ ]:
new_distances = {}
cent = newsys["TAR:0"].centroid
for fragid, frag in newsys.items():
    new_distances[fragid] = norm(array(frag.centroid) - array(cent))

In [ ]:
from BigDFT.Fragments import pairwise_distance
from BigDFT.Atoms import AU_to_A

subsystems = {}

for cut in cutoffs:
    subsystems[cut] = System()
    for i, fragid in enumerate(sorted(list(new_distances), key=new_distances.get)):
        if i >= cut:
            break
        subsystems[cut][fragid] = newsys[fragid]

Write to file.

In [ ]:
from BigDFT.IO import write_pdb

for cut, subsys in subsystems.items():
    with open(join(outdir, "2CzPN_" + str(cut) + ".pdb"), "w") as ofile:
        write_pdb(subsys, ofile)